In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv(r'dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
# # print(df)
# test_sample = df.sample(n=3, random_state=1)

In [3]:
df

,label,message,diff
0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...
10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [4]:
df = df.rename(columns={'message':'text'})
train = {}
test  = df

In [5]:
from commitfit import get_templated_dataset, sample_dataset
from datasets import Dataset, load_metric

In [6]:
# train = df.rename(columns={'3_labels':'label','comment':'text'})

In [7]:
dummy_dataset = Dataset.from_dict(train)
test_dataset = Dataset.from_pandas(test)
dummy_dataset

Dataset({
    features: [],
    num_rows: 0
})

In [44]:
train_dataset = get_templated_dataset(dummy_dataset, candidate_labels=['negative','positive'], sample_size=8)

In [45]:
# list(train['message'].astype(str).values)
train_dataset[0]

{'text': 'This sentence is negative', 'label': 0}

In [46]:
len(train)

0

In [47]:
# encoded_train = tokenizer(train_dataset['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_train["input_ids"].shape)
# encoded_test = tokenizer(test['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_test["input_ids"].shape)
# # encoded_val = tokenizer(val['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding='max_length')

In [48]:
# encoded_train

In [49]:
# train['label'].value_counts()

In [50]:
test['label'].value_counts()

label
0    6347
1    3765
Name: count, dtype: int64

In [51]:
# huggingface-cli login
# train_dataset = CommitDataset(encoded_train, list(train['3_labels']))
# test_dataset = CommitDataset(encoded_test, list(test['3_labels']))
# val_dataset = CommitDataset(encoded_val, list(val['label']))

In [52]:
len(train_dataset)

16

In [53]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [54]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 16
})

In [55]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [56]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [57]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [58]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 1701.89it/s]
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [00:02<00:00,  2.52s/it]
***** Running evaluation *****


CPU times: user 52.6 s, sys: 6.59 s, total: 59.2 s
Wall time: 14.1 s


{'accuracy': 0.44699367088607594,
 'precision': 0.46671118196711786,
 'recall': 0.44699367088607594,
 'f1': 0.4546067616236538}